In [1]:
%load_ext jupyternotify

# import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
# import timeit
import main

from tqdm.auto import tqdm
# from scipy.stats import *

pd.set_option('display.max_columns', None)

<IPython.core.display.Javascript object>

In [45]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [3]:
all_matchUp_df = pd.read_csv('data/PA_data_15-19.csv', index_col=[0])

In [4]:
all_matchUp_df.reset_index(inplace=True)
all_matchUp_df.rename(columns={'index':'pa_index'}, inplace=True)

# Game Env Data:

In [5]:
game_env_df = pd.read_csv('data/GameData_15-22.csv', index_col=[0])

In [6]:
game_env_df.head()

,game_pk,game_num,game_type,official_date,away_team,away_team_abb,home_team,home_team_abb,weather_cond,temp,wind,ump_id,ump_name,wind_speed,wind_direction
0,663451,1,R,2022-10-05,tor,TOR,bal,BAL,Overcast,59,"10 mph, In From LF",427344,Bill Miller,10,In From LF
1,663352,1,R,2022-10-05,tba,TB,bos,BOS,Drizzle,57,"15 mph, In From LF",503586,Tripp Gibson,15,In From LF
2,663272,2,R,2022-10-05,tor,TOR,bal,BAL,Overcast,59,"8 mph, In From LF",427286,Brian Knight,8,In From LF
3,663131,1,R,2022-10-05,chn,CHC,cin,CIN,Sunny,76,"4 mph, Out To CF",605674,Ryan Wills,4,Out To CF
4,663095,1,R,2022-10-05,kca,KC,cle,CLE,Sunny,68,"4 mph, L To R",605670,Dan Merzel,4,L To R


### merge game env with matchup data

In [7]:
matchup_merged_df = all_matchUp_df.merge(game_env_df[['game_pk', 'weather_cond', 'temp', 'ump_id', 'ump_name', 'wind_speed', 'wind_direction']], on='game_pk', how='left')

In [8]:
# determine roof closed or open air
matchup_merged_df['roof_cond'] = 'OpenAir'
matchup_merged_df.loc[matchup_merged_df['weather_cond'].isin(['Dome', 'Roof Closed']), 'roof_cond'] = 'RoofClosed'

In [9]:
matchup_merged_df.head()

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other,weather_cond,temp,ump_id,ump_name,wind_speed,wind_direction,roof_cond
0,0,2015,413649,1,433587,594777,SEA,L,R,K,Kole Calhoun called out on strikes.,0,0,0,0,0,0,1,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir
1,1,2015,413649,2,433587,545361,SEA,R,R,HR,Mike Trout homers (1) on a fly ball to center ...,0,0,0,1,0,0,0,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir
2,2,2015,413649,3,433587,405395,SEA,R,R,K,Albert Pujols strikes out on a foul tip.,0,0,0,0,0,0,1,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir
3,3,2015,413649,4,433587,459964,SEA,L,R,K,Matt Joyce strikes out swinging.,0,0,0,0,0,0,1,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir
4,4,2015,413649,5,450308,457706,SEA,R,R,FO,"Austin Jackson grounds out, shortstop Erick Ay...",0,0,0,0,1,0,0,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir


# Park Factor

In [10]:
pf_df = pd.read_csv('data/ParkFactors_full_15-22.csv', index_col=[0])

In [11]:
pf_df.rename(columns={
    'R':'R_pf',
    'OBP':'OBP_pf',
    'H':'H_pf',
    '1B':'1B_pf',
    '2B':'2B_pf',
    '3B':'3B_pf',
    'HR':'HR_pf',
    'BB':'BB_pf',
    'SO':'K_pf'
}, inplace=True)

In [12]:
pf_df

,Rank,team,venue,year,park_factor,wOBACon,xwOBACon,BACON,xBACON,HardHit,R_pf,OBP_pf,H_pf,1B_pf,2B_pf,3B_pf,HR_pf,BB_pf,K_pf,PA,bat_side,condition,rolling_3yrs
0,1,Rockies,Coors Field,2022,115,113,103,110,103,107,132,110,118,111,125,143.0,143,95,84,"6,112",Both,All,no
1,2,Reds,Great American Ball Park,2022,109,112,101,108,101,101,119,106,107,102,105,70.0,143,107,102,"6,244",Both,All,no
2,3,Red Sox,Fenway Park,2022,107,108,103,107,102,103,114,104,110,105,113,261.0,118,93,96,"6,225",Both,All,no
3,4,Dodgers,Dodger Stadium,2022,104,107,98,105,98,97,108,101,103,98,104,94.0,125,94,106,"6,260",Both,All,no
4,5,Phillies,Citizens Bank Park,2022,104,105,99,103,99,102,108,101,102,99,102,116.0,120,95,98,"6,754",Both,All,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3022,25,Giants,Oracle Park,2015,95,95,100,96,99,108,90,97,93,94,94,156.0,75,115,108,"3,317",R,OpenAir,no
3023,26,Mariners,T-Mobile Park,2015,95,95,99,96,99,94,90,96,92,95,86,80.0,89,101,108,"3,313",R,OpenAir,no
3024,27,Yankees,Yankee Stadium,2015,93,94,101,95,100,100,86,94,91,96,70,53.0,102,97,110,"2,725",R,OpenAir,no
3025,28,Mets,Citi Field,2015,91,91,95,91,97,90,83,92,91,92,89,98.0,88,101,105,"3,722",R,OpenAir,no


In [13]:
# add team abbreviation

team_name_dict = {'Rockies': 'COL',
 'Guardians': 'CLE',
 'Red Sox': 'BOS',
 'Rangers': 'TEX',
 'Orioles': 'BAL',
 'D-backs': 'ARI',
 'Brewers': 'MIL',
 'Twins': 'MIN',
 'Cubs': 'CHC',
 'Reds': 'CIN',
 'Astros': 'HOU',
 'Royals': 'KC',
 'Phillies': 'PHI',
 'Nationals': 'WSH',
 'Cardinals': 'STL',
 'Padres': 'SD',
 'Yankees': 'NYY',
 'White Sox': 'CWS',
 'Blue Jays': 'TOR',
 'Pirates': 'PIT',
 'Rays': 'TB',
 'Dodgers': 'LAD',
 'Tigers': 'DET',
 'Mariners': 'SEA',
 'Angels': 'LAA',
 'Athletics': 'OAK',
 'Marlins': 'MIA',
 'Mets': 'NYM',
 'Braves': 'ATL',
 'Giants': 'SF'}
pf_df['team_abb'] = pf_df['team'].apply(lambda x: team_name_dict[x])

# Merge

In [14]:
# y = all_matchUp_df[main.EVENT_LIST].values
# y_pred = all_matchUp_df[main.PROBA_LIST]
# main.get_brier_score(y, y_pred)

In [15]:
all_matchUp_df.shape

(1354702, 19)

In [16]:
all_matchUp_df.head()

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other
0,0,2015,413649,1,433587,594777,SEA,L,R,K,Kole Calhoun called out on strikes.,0,0,0,0,0,0,1,0
1,1,2015,413649,2,433587,545361,SEA,R,R,HR,Mike Trout homers (1) on a fly ball to center ...,0,0,0,1,0,0,0,0
2,2,2015,413649,3,433587,405395,SEA,R,R,K,Albert Pujols strikes out on a foul tip.,0,0,0,0,0,0,1,0
3,3,2015,413649,4,433587,459964,SEA,L,R,K,Matt Joyce strikes out swinging.,0,0,0,0,0,0,1,0
4,4,2015,413649,5,450308,457706,SEA,R,R,FO,"Austin Jackson grounds out, shortstop Erick Ay...",0,0,0,0,1,0,0,0


In [17]:
matchup_merged_df = matchup_merged_df.merge(
    pf_df[['year', 'bat_side', 'team_abb', 'condition', '1B_pf', '2B_pf', '3B_pf', 'HR_pf', 'BB_pf', 'K_pf']], 
    how='left', left_on=['game_year', 'home_team', 'stand', 'roof_cond'], right_on=['year', 'team_abb', 'bat_side', 'condition']
).copy()

In [18]:
matchup_merged_df[matchup_merged_df['1B_pf'].isna()]

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other,weather_cond,temp,ump_id,ump_name,wind_speed,wind_direction,roof_cond,year,bat_side,team_abb,condition,1B_pf,2B_pf,3B_pf,HR_pf,BB_pf,K_pf
26091,26091,2015,413994,1,501957,430203,BAL,L,R,FO,David DeJesus grounds out to first baseman Chr...,0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26092,26092,2015,413994,2,501957,519306,BAL,R,R,K,Steven Souza Jr. strikes out swinging.,0,0,0,0,0,0,1,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26093,26093,2015,413994,3,501957,452678,BAL,L,R,FO,Asdrubal Cabrera lines out sharply to left fie...,0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26094,26094,2015,413994,4,517008,457477,BAL,L,R,K,Alejandro De Aza strikes out swinging.,0,0,0,0,0,0,1,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26095,26095,2015,413994,5,517008,517370,BAL,L,R,FO,"Jimmy Paredes grounds out, first baseman James...",0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965666,965666,2020,631341,58,445213,645302,MIA,R,R,FO,Victor Robles flies out to center fielder Magn...,0,0,0,0,1,0,0,0,Cloudy,81,484499,Manny Gonzalez,2,R To L,OpenAir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965667,965667,2020,631341,59,445213,607208,MIA,R,R,FO,"Trea Turner grounds out, third baseman Brian A...",0,0,0,0,1,0,0,0,Cloudy,81,484499,Manny Gonzalez,2,R To L,OpenAir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965668,965668,2020,631341,60,445213,594809,MIA,L,R,1B,Adam Eaton singles on a sharp line drive to ri...,1,0,0,0,0,0,0,0,Cloudy,81,484499,Manny Gonzalez,2,R To L,OpenAir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965669,965669,2020,631341,61,445213,665742,MIA,L,R,BB,Juan Soto walks. Adam Eaton to 2nd.,0,0,0,0,0,1,0,0,Cloudy,81,484499,Manny Gonzalez,2,R To L,OpenAir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
matchup_replace_df = matchup_merged_df[matchup_merged_df['1B_pf'].isna()].merge(
    pf_df[pf_df['condition']=='All'][['year', 'bat_side', 'team_abb', 'condition', '1B_pf', '2B_pf', '3B_pf', 'HR_pf', 'BB_pf', 'K_pf']],
    how='left', left_on=['game_year', 'home_team', 'stand'], right_on=['year', 'team_abb', 'bat_side'],
    suffixes=('', '_y')
).copy()

In [20]:
matchup_replace_df.head()

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other,weather_cond,temp,ump_id,ump_name,wind_speed,wind_direction,roof_cond,year,bat_side,team_abb,condition,1B_pf,2B_pf,3B_pf,HR_pf,BB_pf,K_pf,year_y,bat_side_y,team_abb_y,condition_y,1B_pf_y,2B_pf_y,3B_pf_y,HR_pf_y,BB_pf_y,K_pf_y
0,26091,2015,413994,1,501957,430203,BAL,L,R,FO,David DeJesus grounds out to first baseman Chr...,0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,L,BAL,All,107,87,53.0,141,89,96
1,26092,2015,413994,2,501957,519306,BAL,R,R,K,Steven Souza Jr. strikes out swinging.,0,0,0,0,0,0,1,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,R,BAL,All,107,98,72.0,133,92,92
2,26093,2015,413994,3,501957,452678,BAL,L,R,FO,Asdrubal Cabrera lines out sharply to left fie...,0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,L,BAL,All,107,87,53.0,141,89,96
3,26094,2015,413994,4,517008,457477,BAL,L,R,K,Alejandro De Aza strikes out swinging.,0,0,0,0,0,0,1,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,L,BAL,All,107,87,53.0,141,89,96
4,26095,2015,413994,5,517008,517370,BAL,L,R,FO,"Jimmy Paredes grounds out, first baseman James...",0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,L,BAL,All,107,87,53.0,141,89,96


In [21]:
replace_col = ['year', 'bat_side', 'team_abb', 'condition'] + main.PF_LIST
replace_col

['year',
 'bat_side',
 'team_abb',
 'condition',
 '1B_pf',
 '2B_pf',
 '3B_pf',
 'HR_pf',
 'BB_pf',
 'K_pf']

In [22]:
replace_col_y = [y + '_y' for y in replace_col]
replace_col_y

['year_y',
 'bat_side_y',
 'team_abb_y',
 'condition_y',
 '1B_pf_y',
 '2B_pf_y',
 '3B_pf_y',
 'HR_pf_y',
 'BB_pf_y',
 'K_pf_y']

In [23]:
matchup_replace_df[replace_col] = matchup_replace_df[replace_col_y]
# drop _y columns
matchup_replace_df.drop(columns=replace_col_y, inplace=True)

In [24]:
matchup_replace_df.head()

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other,weather_cond,temp,ump_id,ump_name,wind_speed,wind_direction,roof_cond,year,bat_side,team_abb,condition,1B_pf,2B_pf,3B_pf,HR_pf,BB_pf,K_pf
0,26091,2015,413994,1,501957,430203,BAL,L,R,FO,David DeJesus grounds out to first baseman Chr...,0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,2015,L,BAL,All,107,87,53.0,141,89,96
1,26092,2015,413994,2,501957,519306,BAL,R,R,K,Steven Souza Jr. strikes out swinging.,0,0,0,0,0,0,1,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,2015,R,BAL,All,107,98,72.0,133,92,92
2,26093,2015,413994,3,501957,452678,BAL,L,R,FO,Asdrubal Cabrera lines out sharply to left fie...,0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,2015,L,BAL,All,107,87,53.0,141,89,96
3,26094,2015,413994,4,517008,457477,BAL,L,R,K,Alejandro De Aza strikes out swinging.,0,0,0,0,0,0,1,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,2015,L,BAL,All,107,87,53.0,141,89,96
4,26095,2015,413994,5,517008,517370,BAL,L,R,FO,"Jimmy Paredes grounds out, first baseman James...",0,0,0,0,1,0,0,0,Dome,72,429805,Todd Tichenor,0,None,RoofClosed,2015,L,BAL,All,107,87,53.0,141,89,96


In [25]:
replace_pa_index = matchup_replace_df['pa_index'].to_list()
matchup_merged_df.iloc[replace_pa_index] = matchup_replace_df.copy()

In [26]:
matchup_merged_df[matchup_merged_df['1B_pf'].isna()]

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other,weather_cond,temp,ump_id,ump_name,wind_speed,wind_direction,roof_cond,year,bat_side,team_abb,condition,1B_pf,2B_pf,3B_pf,HR_pf,BB_pf,K_pf


In [27]:
# # manually impute 3B_pf for RHB at MIN in 2020 
# matchup_merged_df.loc[matchup_merged_df['3B_pf'].isna(), '3B_pf'] = 282.0

In [28]:
matchup_merged_df['platoon'] = True
matchup_merged_df.loc[matchup_merged_df['stand']==matchup_merged_df['p_throws'], 'platoon'] = False

# Speed

In [29]:
speed_df = pd.read_csv('data/sprint_speed/player_speed.csv', index_col=[0])

In [30]:
matchup_merged_df = matchup_merged_df.merge(speed_df[['player_id', 'year', 'sprint_speed']], 
                     left_on=['batter', 'game_year'], right_on=['player_id', 'year'], how='left')

In [31]:
avg_speed_df = speed_df.groupby(['year'])['sprint_speed'].mean().reset_index()
avg_speed_df

,year,sprint_speed
0,2015,26.540444
1,2016,26.576901
2,2017,26.709552
3,2018,26.713493
4,2019,26.676358
5,2020,26.763527
6,2021,26.735757
7,2022,27.055962


In [32]:
get_speed = lambda x: avg_speed_df[avg_speed_df['year']==x]['sprint_speed'].values[0]

In [33]:
matchup_merged_df.loc[matchup_merged_df['sprint_speed'].isna(), 'sprint_speed'] = matchup_merged_df[matchup_merged_df['sprint_speed'].isna()]['game_year'].apply(get_speed)

In [34]:
matchup_merged_df.drop(columns=['player_id', 'year_x', 'year_y', 'team_abb'], inplace=True)

In [37]:
matchup_merged_df

,pa_index,game_year,game_pk,at_bat_number,pitcher,batter,home_team,stand,p_throws,event_type,des,1B,2B,3B,HR,FO,BB,K,other,weather_cond,temp,ump_id,ump_name,wind_speed,wind_direction,roof_cond,bat_side,condition,1B_pf,2B_pf,3B_pf,HR_pf,BB_pf,K_pf,platoon,sprint_speed
0,0,2015,413649,1,433587,594777,SEA,L,R,K,Kole Calhoun called out on strikes.,0,0,0,0,0,0,1,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir,L,OpenAir,100.0,85.0,77.0,95.0,98.0,109.0,True,26.5
1,1,2015,413649,2,433587,545361,SEA,R,R,HR,Mike Trout homers (1) on a fly ball to center ...,0,0,0,1,0,0,0,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir,R,OpenAir,95.0,86.0,80.0,89.0,101.0,108.0,False,29.7
2,2,2015,413649,3,433587,405395,SEA,R,R,K,Albert Pujols strikes out on a foul tip.,0,0,0,0,0,0,1,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir,R,OpenAir,95.0,86.0,80.0,89.0,101.0,108.0,False,23.3
3,3,2015,413649,4,433587,459964,SEA,L,R,K,Matt Joyce strikes out swinging.,0,0,0,0,0,0,1,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir,L,OpenAir,100.0,85.0,77.0,95.0,98.0,109.0,True,26.3
4,4,2015,413649,5,450308,457706,SEA,R,R,FO,"Austin Jackson grounds out, shortstop Erick Ay...",0,0,0,0,1,0,0,0,Cloudy,54,427520,Larry Vanover,1,Varies,OpenAir,R,OpenAir,95.0,86.0,80.0,89.0,101.0,108.0,False,28.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354697,1354697,2022,663463,75,643316,669304,BAL,R,L,FO,"Jose Miranda grounds out, second baseman Rougn...",0,0,0,0,1,0,0,0,Partly Cloudy,67,484198,Alan Porter,11,R To L,OpenAir,R,OpenAir,119.0,85.0,63.0,77.0,106.0,94.0,True,26.3
1354698,1354698,2022,663463,76,664054,602104,BAL,R,R,1B,Ramon Urias singles on a ground ball to center...,1,0,0,0,0,0,0,0,Partly Cloudy,67,484198,Alan Porter,11,R To L,OpenAir,R,OpenAir,119.0,85.0,63.0,77.0,106.0,94.0,False,26.5
1354699,1354699,2022,663463,77,664054,621532,BAL,L,R,K,Anthony Bemboom called out on strikes.,0,0,0,0,0,0,1,0,Partly Cloudy,67,484198,Alan Porter,11,R To L,OpenAir,L,OpenAir,112.0,86.0,67.0,77.0,89.0,99.0,True,26.9
1354700,1354700,2022,663463,78,664054,622761,BAL,R,R,FO,Jorge Mateo flies out sharply to center fielde...,0,0,0,0,1,0,0,0,Partly Cloudy,67,484198,Alan Porter,11,R To L,OpenAir,R,OpenAir,119.0,85.0,63.0,77.0,106.0,94.0,False,30.1


In [51]:
matchup_merged_df['1B_pf'].describe()

count   1354702.00000
mean        100.21879
std           7.95613
min          51.00000
25%          95.00000
50%         100.00000
75%         105.00000
max         153.00000
Name: 1B_pf, dtype: float64

In [52]:
matchup_merged_df['2B_pf'].describe()

count   1354702.00000
mean        100.92256
std          16.98298
min           0.00000
25%          89.00000
50%          99.00000
75%         110.00000
max         169.00000
Name: 2B_pf, dtype: float64

In [38]:
# matchup_merged_df.to_csv('data/PA_data_merged_15-22_012323.csv')

In [49]:
matchup_merged_df['roof_cond'].unique()

array(['OpenAir', 'RoofClosed'], dtype=object)